In [9]:
import pandas as pd
import numpy as np
import torch

In [11]:
# 1부 대본 데이터 불러오기
script1 = pd.read_excel('../data/야인시대 1부 (1~50) 통합대본.xlsx')
script1 = script1.dropna()
script1.head()

,회차,시간,인물,대사
0,1,0~5,# 상황,비오는 밤길을 달려가는 차량. 헤드라이트 클로즈업 및 차창 밖으로 비가 내리고 있는...
1,1,0~5,# 상황,비가 떨어지는 자동차 바닥 아래로 스타워즈 오프닝 스타일의 자막 흘러감.
2,1,0~5,나레이션,이 이야기는 암울했던 민족의 수난기와 격동기의 역사를 살다 갔던 영원한 야인 김두한...
3,1,0~5,나레이션,그러나 드라마의 원만한 진행을 위하여 시대와 역사적 상황을 운영하는 인물 일부분에서...
4,1,0~5,# 장소,어느 도예촌 전경


In [12]:
# 2부 대본 데이터 불러오기
script2 = pd.read_excel('../data/야인시대 2부(51~124) 통합대본.xlsx')
script2 = script2.dropna()
script2.head()

,회차,시간,인물,대사
0,51,0~5,# 장소,"절간에 앉아있는 김두한, 불상 등 클로즈업"
1,51,0~5,# 두한,고민가득한 표정 줌인
2,51,0~5,두한,해방이다. 그토록 온 세상이 염원하던 독립이 왔다. 이 독립을 위해 얼마나 많은 사...
3,51,0~5,두한,아버님이 그리하셨고 어머님이 그리하셨다. 이제 난 이 새로운 조국에서 과연 무엇을 ...
4,51,0~5,두한,주먹패로 살아간다는 것은 더 이상 의미가 없다. 무엇을 위해 누구와 싸운단 말인가?*


In [13]:
# 데이터 합치기
script = pd.concat([script1, script2])
script

,회차,시간,인물,대사
0,1,0~5,# 상황,비오는 밤길을 달려가는 차량. 헤드라이트 클로즈업 및 차창 밖으로 비가 내리고 있는...
1,1,0~5,# 상황,비가 떨어지는 자동차 바닥 아래로 스타워즈 오프닝 스타일의 자막 흘러감.
2,1,0~5,나레이션,이 이야기는 암울했던 민족의 수난기와 격동기의 역사를 살다 갔던 영원한 야인 김두한...
3,1,0~5,나레이션,그러나 드라마의 원만한 진행을 위하여 시대와 역사적 상황을 운영하는 인물 일부분에서...
4,1,0~5,# 장소,어느 도예촌 전경
...,...,...,...,...
35719,124,60~65,# 사물,김두한 영정 클로즈 업
35720,124,60~65,나레이션,김두한. 그는 일제 말 우리가 주권을 잃었던 식민지 시절부터 해방 이후 좌우익의 대...
35721,124,60~65,최동열,나는 오랫동안 자네를 지켜보아 온 사람일세. 자네는 자네답게 살았어. 조선의 주먹 ...
35722,124,60~65,최동열,나름대로 자네의 역사를 가지고 자네의 시대를 치열하고 열심히 살았다는 얘기야. 뭐랄...


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# 대사와 레이블을 포함하는 데이터프레임
df = pd.DataFrame({
    '대사': script['대사'],  # 대사 컬럼
    '레이블': script['레이블']  # 레이블 컬럼.
})

# 훈련 데이터와 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(df['대사'], df['레이블'], test_size=0.2, random_state=42)

# 파이프라인 생성
text_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LinearSVC()),
])

# 모델 훈련
text_clf.fit(X_train, y_train)

# 테스트 데이터에 대한 예측
predictions = text_clf.predict(X_test)

# 성능 보고서 출력
print(classification_report(y_test, predictions))

In [ ]:
# BERT 모델을 사용한 대화 분류
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch

# BERT 토크나이저와 모델 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=6)  # 레이블 수에 맞게 조정

# 데이터셋 정의
class DialogueDataset(Dataset):
    def __init__(self, dialogues, labels, tokenizer, max_len):
        self.dialogues = dialogues
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.dialogues)

    def __getitem__(self, item):
        dialogue = str(self.dialogues[item])
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
            dialogue,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'dialogue_text': dialogue,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# 데이터셋과 데이터로더 생성
max_len = 128
batch_size = 16
dataset = DialogueDataset(df['대사'].to_numpy(), df['레이블'].to_numpy(), tokenizer, max_len)
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)
train_loader = DataLoader(train_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

# 모델 훈련
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss().to(device)

for epoch in range(epochs):
    model.train()
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

# 모델 평가
model.eval()
for batch in test_loader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

    logits = outputs[1]
    predictions = torch.argmax(logits, dim=-1)
    correct_predictions = predictions == labels
    accuracy = correct_predictions.sum().item() / correct_predictions.numel()

print(f'Test Accuracy: {accuracy * 100:.2f}%')